### Python implementation of consignment stock models

The following script implements all four consignment stock (CS) models analysed in this paper under a common set of base-case parameters:

- **CS (k = 0)** – baseline single-vendor, single-buyer CS model of Braglia & Zavanella (2003), without delayed deliveries.
- **CS–k** – baseline CS model with delayed deliveries, where up to *k* shipments per cycle are postponed.
- **Huang–Chen model** – cost-structure extension with separate financial and storage components and a CS–k policy.
- **OCS model** – Persona et al.’s (2007) obsolescence CS model with a finite product lifetime.
- **Multi-buyer CS model** – Zavanella & Zanoni’s (2009) single-vendor, multi-buyer CS formulation.

In [ ]:
import numpy as np
import itertools


# Parameters (base case)

base_params = {
    "A1": 500.0,
    "A2": 50.0,
    "h1": 5.0,
    "h2": 8.0,
    "D": 1000.0,
    "P": 2000.0,
}

hc_params = {
    **base_params,
    "I": 0.10,
    "p1": 40.0,
    "p2": 60.0,
    "h1s": 1.0,
    "h2s": 2.0,
}

ocs_params = {
    **base_params,
    "T": 1.0,
    "p2": 60.0,
}

mb_params = {
    "A1": 500.0,
    "h1": 5.0,
    "P": 2000.0,
    "buyers": [
        {"name": "Buyer 1", "d": 600.0, "A2": 40.0, "h2": 8.0},
        {"name": "Buyer 2", "d": 400.0, "A2": 60.0, "h2": 9.0},
    ],
}


# Baseline CS (with / without k)


def cs_AB(n, k, params):
    A1, A2, h1, h2, D, P = (
        params["A1"], params["A2"], params["h1"], params["h2"],
        params["D"], params["P"]
    )
    A = (A1 + n * A2) * D / n
    B_base = h2 * (D / P + n * (P - D) / (2 * P))
    delay_term = D / (2 * P) + (P - D) / (n * P) * (k + 1) * k / 2.0
    B = B_base - (h2 - h1) * delay_term
    return A, B

def optimise_cs_no_k(params, n_max=10):
    best = None
    for n in range(1, n_max + 1):
        A, B = cs_AB(n, 0, params)
        if B <= 0:
            continue
        q = (A / B) ** 0.5
        cost = 2 * (A * B) ** 0.5
        if best is None or cost < best["cost"]:
            best = {"model": "CS (k=0)", "n": n, "k": 0, "q": q, "cost": cost}
    return best

def optimise_cs_with_k(params, n_max=10):
    best = None
    for n in range(1, n_max + 1):
        for k in range(0, n):          # k = 0,...,n-1
            A, B = cs_AB(n, k, params)
            if B <= 0:
                continue
            q = (A / B) ** 0.5
            cost = 2 * (A * B) ** 0.5
            if best is None or cost < best["cost"]:
                best = {"model": "CS–k", "n": n, "k": k, "q": q, "cost": cost}
    return best

# Cost-structure CS–k (Huang & Chen)

def optimise_huang_chen(params, n_max=10):
    A1, A2, D, P = params["A1"], params["A2"], params["D"], params["P"]
    I, p1, h1s, h2s = params["I"], params["p1"], params["h1s"], params["h2s"]
    Ip1 = I * p1
    best = None

    for n in range(1, n_max + 1):
        k = n - 1 if h2s > h1s else 0
        A = (A1 + n * A2) * D / n
        fin_vendor = Ip1 * (D / P + n * (P - D) / (2 * P))
        delay_coeff = (P - D) / (n * P) * (k + 1) * k / 2.0
        storage_buyer = h2s * (D / (2 * P) + n * (P - D) / (2 * P) - delay_coeff)
        storage_vendor = h1s * (D / (2 * P) + delay_coeff)
        B = fin_vendor + storage_buyer + storage_vendor
        if B <= 0:
            continue
        q = (A / B) ** 0.5
        cost = 2 * (A * B) ** 0.5
        if best is None or cost < best["cost"]:
            best = {"model": "Huang–Chen", "n": n, "k": k, "q": q, "cost": cost}
    return best

# Obsolescence CS

def optimise_ocs(params, n_max=10):
    A1, A2, h1, h2, D, P, T, p2 = (
        params["A1"], params["A2"], params["h1"], params["h2"],
        params["D"], params["P"], params["T"], params["p2"]
    )
    best = None

    for n in range(1, n_max + 1):
        a = A1 + n * A2
        term1 = 0.5 * h1 * T**2 * D**2 / (n * P)
        term2 = 0.5 * h2 * (T**2 * D * (n - 1) / n - T**2 * D**2 / P)
        term3 = p2 * T * D**2 / (n * P)
        B = term1 + term2 + term3
        if B <= 0:
            continue

        k_cont = (B / a) ** 0.5
        if k_cont < 1:
            k_cont = 1.0
        k_candidates = sorted({int(np.floor(k_cont)), int(np.ceil(k_cont))})
        k_candidates = [k for k in k_candidates if k >= 1]

        for k in k_candidates:
            cost = (a * k + B / k) / T
            q = T * D / (n * k)
            if best is None or cost < best["cost"]:
                best = {"model": "OCS", "n": n, "k_cycles": k, "q": q, "cost": cost}
    return best

# Multi-buyer CS (Zavanella & Zanoni)

def optimise_multi_buyer(params, n_max=6):
    A1, h1, P = params["A1"], params["h1"], params["P"]
    buyers = params["buyers"]
    best = None

    for n_tuple in itertools.product(range(1, n_max + 1), repeat=len(buyers)):
        A_total = A1 + sum(n_i * b["A2"] for n_i, b in zip(n_tuple, buyers))
        vendor_term = h1 / (2 * P) * sum((b["d"] ** 2) / n_i for n_i, b in zip(n_tuple, buyers))
        buyer_term = 0.5 * sum(
            b["h2"] * b["d"] * (1 - b["d"] / P + b["d"] / (n_i * P))
            for n_i, b in zip(n_tuple, buyers)
        )
        C_hold = vendor_term + buyer_term
        if C_hold <= 0:
            continue
        T_star = (A_total / C_hold) ** 0.5
        cost = A_total / T_star + C_hold * T_star
        if best is None or cost < best["cost"]:
            best = {
                "model": "Multi-buyer CS",
                "n_vec": n_tuple,
                "T": T_star,
                "cost": cost,
            }

    if best is not None:
        q_vec = [b["d"] * best["T"] / n_i for n_i, b in zip(best["n_vec"], buyers)]
        best["q_vec"] = q_vec
    return best

# Example run

if __name__ == "__main__":
    print(optimise_cs_no_k(base_params))
    print(optimise_cs_with_k(base_params))
    print(optimise_huang_chen(hc_params))
    print(optimise_ocs(ocs_params))
    print(optimise_multi_buyer(mb_params))


{'model': 'CS (k=0)', 'n': 4, 'k': 0, 'q': 124.72191289246472, 'cost': 2806.243040080456}
{'model': 'CS–k', 'n': 6, 'k': 5, 'q': 107.6763804116331, 'cost': 2476.5567494675615}
{'model': 'Huang–Chen', 'n': 5, 'k': 4, 'q': 127.34290799340266, 'cost': 2355.8437978779493}
{'model': 'OCS', 'n': 10, 'k_cycles': 2, 'q': 50.0, 'cost': 4362.5}
{'model': 'Multi-buyer CS', 'n_vec': (2, 1), 'T': 0.3873740009953272, 'cost': 3304.3002284901413, 'q_vec': [116.21220029859816, 154.94960039813088]}
